In [1]:
# import dependencies
import sys
import pandas as pd
import hashlib  #hashing library
import time   #produce time stamp
import json   
import requests #request info from the API

In [2]:
def hash_gen():
    #Generating the Hash
    m = hashlib.md5()

    ts = str(time.time())   #creates time stamp as string
    ts_byte = bytes(ts, 'utf-8')  #converts the timestamp into a byte 
    m.update(ts_byte)  # add the timestamp to hash
    m.update(b"8c6e53313bb343e03a9b8d2a6d3da0754415a07b") #add the private key to 
        #the hash in byte format
    m.update(b"6040b6d7af5b870e8309d4fbd15a88d2") #add the public key to 
        #the hash in byte format
    hasht = m.hexdigest()    #Marvel requires the string to be in hex(mentioned nowhere)
    return hasht

In [109]:
def API_call(api_key, hasht, nameStartsWith):
    #constructing the query

    base_url = "https://gateway.marvel.com"  #base url
    query = "/v1/public/characters" +"?"  #Query for all characters

    #Actual query look like:
    #query_url = base_url + query +"nameStartsWith=spider&limit=10&"+"ts=" + ts+ "&apikey=" + api_key + "&hash=" + hasht
    #print(query_url) 
    
    query_url = base_url + query
    ts = str(time.time())
    payload = {
        'nameStartsWith':nameStartsWith,
        'limit':100,
        'ts':ts,
        'apikey':api_key,
        'hash':hasht
    } # all the parameters for the query

    #Making the API request and receiving info back as a json
    data = requests.get(query_url, params=payload).json()
    # json_obj = json.dumps(data, indent=4) # can be further stored as json dump

    try:
        #Storing the data to a dataframe
        df = pd.DataFrame(data)
        df_norm = pd.json_normalize(df["data"]["results"])  #Breaks down keys to the basic form
        df_select = df_norm[["id", "name", "comics.available", "series.available", "stories.available", "events.available"]]
        print(nameStartsWith, "queried characters: ", data["data"]["count"])
        return df_select
    
    except:
        if data["code"] == 200:
            print(nameStartsWith, "queried characters: 0")
        else:
            print(data)

In [118]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2" 
df_9 = API_call(api_key, hash_gen(), '9')

9 queried characters: 0


In [119]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2" 
df_3 = API_call(api_key, hash_gen(), '3')

3 queried characters:  1


In [121]:
pd.concat([df_9, df_3])

,id,name,comics.available,series.available,stories.available,events.available
0,1011334,3-D Man,12,3,21,1


In [122]:
def All_calls(api_key):
    #All marvel characters fetched by going through every ascii characters
    i = 0
    df_final = pd.DataFrame() 
    while i<129:
        df_select = API_call(api_key, hash_gen(), chr(i))
        df_final = pd.concat([df_final, df_select])   #appending every call data
        i=i+1
    return df_final

In [123]:
api_key = "6040b6d7af5b870e8309d4fbd15a88d2" 
df_final = All_calls(api_key)

  queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
	 queried characters: 0

 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
 queried characters: 0
  queried characters: 0
! queried characters: 0
{'code': 'InvalidCredentials', 'message': 'That hash, timestamp and key combination is invalid.'}
# queried characters: 0
$ queried characters: 0
% queried characters:  100
& queried ch

In [18]:
df_final.head()

,id,name,comics.available,series.available,stories.available,events.available
0,1011334,3-D Man,12,3,21,1
1,1017100,A-Bomb (HAS),4,2,7,0
2,1009144,A.I.M.,51,34,54,0
3,1010699,Aaron Stack,14,3,27,0
4,1009146,Abomination (Emil Blonsky),55,27,65,1


In [125]:
df_final.shape

(2974, 6)

In [126]:
df_rm_dup = df_final.drop_duplicates(subset=["id"]) #remove duplicates

In [127]:
df_rm_dup.shape

(1399, 6)

In [128]:
df_rm_dup.to_csv("./marvel_data.csv", index=False) #saving the dataset

# MARVEL FILTER

In [17]:
df = pd.read_csv("./marvel_data.csv")   #opening the dataset

In [18]:
df.head()

,Unnamed: 0,id,name,comics.available,series.available,stories.available,events.available
0,0,1011334,3-D Man,12,3,21,1
1,1,1017100,A-Bomb (HAS),4,2,7,0
2,2,1009144,A.I.M.,51,34,54,0
3,3,1010699,Aaron Stack,14,3,27,0
4,4,1009146,Abomination (Emil Blonsky),55,27,65,1


In [19]:
def filter(df, op_column, op, value, display_column):
    #make sql like where queries by passing in the column, value , comparision operator
    #and select the columns you want to be displayed
    df_show = pd.DataFrame()
    match op:
        case '>':
            df_show = df[df[op_column]>value][display_column]
            return df_show
        case '<':
            df_show = df[df[op_column]<value][display_column]
            return df_show
        case '=':
            df_show = df[df[op_column]==value][display_column]
            return df_show
        case '>=':
            df_show = df[df[op_column]>=value][display_column]
            return df_show
        case '<=':
            df_show = df[df[op_column]<=value][display_column]
            return df_show

SyntaxError: invalid syntax (<ipython-input-19-bdf780bf44b4>, line 5)

In [48]:
df_show = filter(df, "comics.available", '=', 4, ["id", "name"])

In [49]:
df_show

,id,name
1,1017100,A-Bomb (HAS)
19,1011136,Air-Walker (Gabriel Lan)
20,1011176,Ajak
41,1010673,Amphibian (Earth-712)
93,1011354,Barracuda
195,1011139,Cammi
219,1010143,Caretaker
250,1012065,Chthon
257,1011286,Cobalt Man
279,1014991,Crusher Hogan (Ultimate)


In [166]:
def filter(df, size, op_column, op, value, display_column):
    #make sql like where queries by passing in the column, value , comparision operator
    #and select the columns you want to be displayed
    df_show = pd.DataFrame()
    match op:
        case '>':
            df_show = df[df[op_column]>value][display_column]
            return df_show
        case '<':
            df_show = df[df[op_column]<value][display_column]
            return df_show
        case '=':
            df_show = df[df[op_column]==value][display_column]
            return df_show
        case '>=':
            df_show = df[df[op_column]>=value][display_column]
            return df_show
        case '<=':
            df_show = df[df[op_column]<=value][display_column]
            return df_show

In [169]:
df_show = filter(df_final, 2, ["comics.available", "name"], ['=', '='], [4, 'Aaron Stack'], ["id", "name"])

TypeError: filter() takes 5 positional arguments but 6 were given